In [ ]:
import pandas as pd
import numpy as np
import sys, os
sys.path.append(os.path.abspath(".."))

import src.preprocessing
from importlib import reload
reload(src.preprocessing)

from src.preprocessing import (
    aggregate_user_day_activity, 
    add_rolling_averages,
    compute_cancellation_batch
)

# Import sklearn components
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import lightgbm as lgb
from sklearn.metrics import classification_report, roc_auc_score

In [ ]:
# ============================================================================
# SIMPLIFIED CUSTOM TRANSFORMERS FOR PIPELINE
# ============================================================================

# First, reload the updated preprocessing module
import src.preprocessing
from importlib import reload
reload(src.preprocessing)
from src.preprocessing import compute_cancellation_batch

class CancellationTargetTransformer(BaseEstimator, TransformerMixin):
    """
    Efficiently computes cancellation targets using vectorized operations.
    Must be provided with raw_df during __init__.
    """
    def __init__(self, window_days=10, raw_df=None):
        self.window_days = window_days
        self.raw_df = raw_df
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        if self.raw_df is None:
            raise ValueError("raw_df must be provided")
        
        print(f"Computing churn targets (vectorized, window={self.window_days}d)...")
        
        # Use efficient batch computation
        churn_targets = compute_cancellation_batch(
            self.raw_df,
            X,
            window_days=self.window_days
        )
        
        # Merge with X
        X_copy = X.copy()
        X_copy['date'] = pd.to_datetime(X_copy['date'])
        churn_targets['date'] = pd.to_datetime(churn_targets['date'])
        X_copy['userId'] = X_copy['userId'].astype(int)
        churn_targets['userId'] = churn_targets['userId'].astype(int)
        
        result = X_copy.merge(churn_targets, on=['userId', 'date'], how='left')
        
        print(f"Churn status distribution:\n{result['churn_status'].value_counts()}")
        return result


class RollingAverageTransformer(BaseEstimator, TransformerMixin):
    """Computes rolling average features."""
    def __init__(self, columns=None, window_days=7):
        self.columns = columns if columns is not None else ['NextSong']
        self.window_days = window_days
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        print(f"Computing rolling averages (window={self.window_days}d)...")
        return add_rolling_averages(X, columns=self.columns, n=self.window_days)


class ThumbsRatioTransformer(BaseEstimator, TransformerMixin):
    """Computes thumbs ratio from rolling averages."""
    def __init__(self, window_days=7):
        self.window_days = window_days
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        up_col = f'thumbs_up_avg_{self.window_days}d'
        down_col = f'thumbs_down_avg_{self.window_days}d'
        ratio_col = f'thumbs_ratio_{self.window_days}d'
        
        if up_col in X_copy.columns and down_col in X_copy.columns:
            denominator = X_copy[up_col] + X_copy[down_col]
            X_copy[ratio_col] = X_copy[up_col] / denominator.replace(0, np.nan)
            X_copy[ratio_col] = X_copy[ratio_col].fillna(0)
        
        return X_copy


class TrendFeaturesTransformer(BaseEstimator, TransformerMixin):
    """Creates trend features by comparing short-term (7d) vs long-term (14d) averages."""
    def __init__(self, columns=None):
        self.columns = columns if columns is not None else ['NextSong']
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        
        # For each column, compute trend (7d avg / 14d avg - 1)
        # Positive = increasing activity, Negative = decreasing activity
        for col in self.columns:
            col_7d = f'{col.lower().replace(" ", "_")}_avg_7d'
            col_14d = f'{col.lower().replace(" ", "_")}_avg_14d'
            trend_col = f'{col.lower().replace(" ", "_")}_trend'
            
            if col_7d in X_copy.columns and col_14d in X_copy.columns:
                # Compute ratio: (7d / 14d) - 1
                # This gives % change: positive = increasing, negative = decreasing
                denominator = X_copy[col_14d].replace(0, np.nan)
                X_copy[trend_col] = (X_copy[col_7d] / denominator) - 1
                X_copy[trend_col] = X_copy[trend_col].fillna(0)
        
        return X_copy


class FeaturePreprocessor(BaseEstimator, TransformerMixin):
    """Handles type conversions and missing value imputation."""
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        
        # Convert level to binary
        if 'level' in X_copy.columns:
            X_copy['level'] = (X_copy['level'] == 'paid').astype(int)
        
        # Create user lifecycle feature: long-time user (30+ days) vs recent user
        if 'days_since_registration' in X_copy.columns:
            X_copy['is_established_user'] = (X_copy['days_since_registration'] >= 30).astype(int)
        
        # Fill ratio columns with 0
        ratio_cols = [col for col in X_copy.columns if 'ratio' in col.lower()]
        for col in ratio_cols:
            if col in X_copy.columns:
                X_copy[col] = pd.to_numeric(X_copy[col], errors='coerce').fillna(0)
        
        return X_copy

In [ ]:
root = '/Users/mdiaspinto/Documents/School/Python Data Science/Final Project/kaggle-churn'
df_raw = pd.read_parquet(root + '/data/train.parquet')

# Clean up: convert object columns to category, drop unnecessary columns
object_cols = df_raw.select_dtypes(include="object").columns
df_raw[object_cols] = df_raw[object_cols].astype("category")
df_raw = df_raw.drop(columns=['gender', 'firstName', 'lastName', 'location', 'userAgent'])

print(f"Raw data shape: {df_raw.shape}")
print(f"Date range: {pd.to_datetime(df_raw['time']).min()} to {pd.to_datetime(df_raw['time']).max()}")

In [ ]:
# Aggregate to user-day level
print("\nAggregating events to user-day level...")
df_agg = aggregate_user_day_activity(df_raw)
df_agg['userId'] = df_agg['userId'].astype(int)

print(f"Aggregated data shape: {df_agg.shape}")
print(f"Date range: {df_agg['date'].min()} to {df_agg['date'].max()}")

In [ ]:
# Temporal train-test split
print("\n" + "=" * 60)
print("TEMPORAL TRAIN/TEST SPLIT")
print("=" * 60)

cutoff_date = pd.to_datetime('2018-11-01')
df_agg['date'] = pd.to_datetime(df_agg['date'])

df_train = df_agg[df_agg['date'] < cutoff_date].copy()
df_test = df_agg[df_agg['date'] >= cutoff_date].copy()

print(f"Training set: {df_train.shape}")
print(f"Test set: {df_test.shape}")
print(f"Train dates: {df_train['date'].min()} to {df_train['date'].max()}")
print(f"Test dates: {df_test['date'].min()} to {df_test['date'].max()}")

In [ ]:
# ============================================================================
# BUILD AND APPLY FEATURE ENGINEERING PIPELINE
# ============================================================================
print("\n" + "=" * 60)
print("FEATURE ENGINEERING PIPELINE (VECTORIZED & EFFICIENT)")
print("=" * 60)

# Define feature columns to track
feature_cols_list = ['Add Friend', 'Add to Playlist', 'Thumbs Down', 'Thumbs Up', 'Error']

# Create pipeline for training data
print("\n1. Transforming TRAINING data...")
train_pipeline = Pipeline([
    ('churn_target', CancellationTargetTransformer(window_days=10, raw_df=df_raw)),
    ('rolling_avg_7d', RollingAverageTransformer(columns=feature_cols_list, window_days=7)),
    ('rolling_avg_14d', RollingAverageTransformer(columns=feature_cols_list, window_days=14)),
    ('thumbs_ratio_7d', ThumbsRatioTransformer(window_days=7)),
    ('thumbs_ratio_14d', ThumbsRatioTransformer(window_days=14)),
    ('trend_features', TrendFeaturesTransformer(columns=feature_cols_list)),
    ('preprocessor', FeaturePreprocessor()),
])

df_train_features = train_pipeline.fit_transform(df_train)

print(f"\nTraining features shape: {df_train_features.shape}")
print(f"Columns: {df_train_features.columns.tolist()[:10]}... (showing first 10)")

# Apply same pipeline to test data
print("\n2. Transforming TEST data...")
test_pipeline = Pipeline([
    ('churn_target', CancellationTargetTransformer(window_days=10, raw_df=df_raw)),
    ('rolling_avg_7d', RollingAverageTransformer(columns=feature_cols_list, window_days=7)),
    ('rolling_avg_14d', RollingAverageTransformer(columns=feature_cols_list, window_days=14)),
    ('thumbs_ratio_7d', ThumbsRatioTransformer(window_days=7)),
    ('thumbs_ratio_14d', ThumbsRatioTransformer(window_days=14)),
    ('trend_features', TrendFeaturesTransformer(columns=feature_cols_list)),
    ('preprocessor', FeaturePreprocessor()),
])

df_test_features = test_pipeline.fit_transform(df_test)

print(f"Test features shape: {df_test_features.shape}")

In [ ]:
# ============================================================================
# EXTRACT FEATURES AND TARGET
# ============================================================================
print("\n" + "=" * 60)
print("EXTRACTING FEATURES AND TARGET")
print("=" * 60)

exclude_cols = ['userId', 'date', 'churn_status']
feature_cols = [col for col in df_train_features.columns if col not in exclude_cols]

X_train = df_train_features[feature_cols].copy()
y_train = df_train_features['churn_status'].copy()

X_test = df_test_features[feature_cols].copy()
y_test = df_test_features['churn_status'].copy()

print(f"\nTraining set:")
print(f"  X_train shape: {X_train.shape}")
print(f"  y_train shape: {y_train.shape}")
print(f"  Churn rate: {y_train.mean():.4f}")
print(f"  Churn distribution:\n{y_train.value_counts()}")

print(f"\nTest set:")
print(f"  X_test shape: {X_test.shape}")
print(f"  y_test shape: {y_test.shape}")
print(f"  Churn rate: {y_test.mean():.4f}")
print(f"  Churn distribution:\n{y_test.value_counts()}")

In [ ]:
# ============================================================================
# TIME-SERIES CROSS-VALIDATION
# ============================================================================
print("\n" + "=" * 60)
print("TIME-SERIES CROSS-VALIDATION")
print("=" * 60)

from sklearn.metrics import balanced_accuracy_score, roc_auc_score, recall_score, precision_score

# Use the full aggregated dataset for walk-forward validation
df_full = df_agg.copy()
df_full['date'] = pd.to_datetime(df_full['date'])

# Define time splits (walk-forward approach)
# Train on expanding window, test on next period
splits = [
    ('2018-10-01', '2018-10-21', '2018-10-22', '2018-10-31'),  # Split 1: 21d train, 10d test
    ('2018-10-01', '2018-10-24', '2018-10-25', '2018-11-03'),  # Split 2: 24d train, 10d test  
    ('2018-10-01', '2018-10-27', '2018-10-28', '2018-11-06'),  # Split 3: 27d train, 10d test
    ('2018-10-01', '2018-10-30', '2018-10-31', '2018-11-09'),  # Split 4: 30d train, 10d test
    ('2018-10-01', '2018-11-02', '2018-11-03', '2018-11-12'),  # Split 5: 33d train, 10d test
]

cv_results = []

for i, (train_start, train_end, test_start, test_end) in enumerate(splits, 1):
    print(f"\n{'='*50}")
    print(f"FOLD {i}: Train [{train_start} to {train_end}], Test [{test_start} to {test_end}]")
    print(f"{'='*50}")
    
    # Create temporal splits
    df_train_cv = df_full[(df_full['date'] >= train_start) & (df_full['date'] <= train_end)].copy()
    df_test_cv = df_full[(df_full['date'] >= test_start) & (df_full['date'] <= test_end)].copy()
    
    print(f"Train size: {len(df_train_cv):,} rows, Test size: {len(df_test_cv):,} rows")
    
    # Build feature pipeline for this fold
    train_pipeline_cv = Pipeline([
        ('churn_target', CancellationTargetTransformer(window_days=10, raw_df=df_raw)),
        ('rolling_avg_7d', RollingAverageTransformer(columns=feature_cols_list, window_days=7)),
        ('rolling_avg_14d', RollingAverageTransformer(columns=feature_cols_list, window_days=14)),
        ('thumbs_ratio_7d', ThumbsRatioTransformer(window_days=7)),
        ('thumbs_ratio_14d', ThumbsRatioTransformer(window_days=14)),
        ('trend_features', TrendFeaturesTransformer(columns=feature_cols_list)),
        ('preprocessor', FeaturePreprocessor()),
    ])
    
    test_pipeline_cv = Pipeline([
        ('churn_target', CancellationTargetTransformer(window_days=10, raw_df=df_raw)),
        ('rolling_avg_7d', RollingAverageTransformer(columns=feature_cols_list, window_days=7)),
        ('rolling_avg_14d', RollingAverageTransformer(columns=feature_cols_list, window_days=14)),
        ('thumbs_ratio_7d', ThumbsRatioTransformer(window_days=7)),
        ('thumbs_ratio_14d', ThumbsRatioTransformer(window_days=14)),
        ('trend_features', TrendFeaturesTransformer(columns=feature_cols_list)),
        ('preprocessor', FeaturePreprocessor()),
    ])
    
    # Transform data
    df_train_features_cv = train_pipeline_cv.fit_transform(df_train_cv)
    df_test_features_cv = test_pipeline_cv.fit_transform(df_test_cv)
    
    # Extract features and target
    exclude_cols = ['userId', 'date', 'churn_status']
    feature_cols_cv = [col for col in df_train_features_cv.columns if col not in exclude_cols]
    
    X_train_cv = df_train_features_cv[feature_cols_cv].copy()
    y_train_cv = df_train_features_cv['churn_status'].copy()
    X_test_cv = df_test_features_cv[feature_cols_cv].copy()
    y_test_cv = df_test_features_cv['churn_status'].copy()
    
    print(f"Churn rate - Train: {y_train_cv.mean():.4f}, Test: {y_test_cv.mean():.4f}")
    
    # Calculate class weight for this fold
    neg_count_cv = (y_train_cv == 0).sum()
    pos_count_cv = (y_train_cv == 1).sum()
    scale_pos_weight_cv = neg_count_cv / pos_count_cv
    
    # Build and train model
    numeric_features_cv = X_train_cv.select_dtypes(include=['int64', 'float64']).columns.tolist()
    categorical_features_cv = X_train_cv.select_dtypes(include=['category', 'object']).columns.tolist()
    
    preprocessor_cv = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numeric_features_cv),
            ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features_cv)
        ]
    )
    
    model_pipeline_cv = Pipeline([
        ('preprocessor', preprocessor_cv),
        ('classifier', lgb.LGBMClassifier(
            random_state=42,
            n_estimators=100,
            verbose=-1,
            scale_pos_weight=scale_pos_weight_cv,
            max_depth=6,
            learning_rate=0.1,
            min_child_weight=1,
            subsample=0.8,
            colsample_bytree=0.8,
            n_jobs=-1
        ))
    ])
    
    # Train and evaluate
    model_pipeline_cv.fit(X_train_cv, y_train_cv)
    y_pred_cv = model_pipeline_cv.predict(X_test_cv)
    y_pred_proba_cv = model_pipeline_cv.predict_proba(X_test_cv)[:, 1]
    
    # Calculate metrics
    bal_acc = balanced_accuracy_score(y_test_cv, y_pred_cv)
    roc_auc = roc_auc_score(y_test_cv, y_pred_proba_cv)
    recall = recall_score(y_test_cv, y_pred_cv)
    precision = precision_score(y_test_cv, y_pred_cv)
    
    cv_results.append({
        'fold': i,
        'train_start': train_start,
        'train_end': train_end,
        'test_start': test_start,
        'test_end': test_end,
        'train_size': len(df_train_cv),
        'test_size': len(df_test_cv),
        'balanced_accuracy': bal_acc,
        'roc_auc': roc_auc,
        'recall': recall,
        'precision': precision,
        'churn_rate_train': y_train_cv.mean(),
        'churn_rate_test': y_test_cv.mean()
    })
    
    print(f"Results: Balanced Acc={bal_acc:.4f}, ROC-AUC={roc_auc:.4f}, Recall={recall:.4f}, Precision={precision:.4f}")

# Summary
print("\n" + "=" * 60)
print("CROSS-VALIDATION SUMMARY")
print("=" * 60)

cv_df = pd.DataFrame(cv_results)
print(cv_df.to_string(index=False))

print(f"\nMean Metrics Across {len(splits)} Folds:")
print(f"  Balanced Accuracy: {cv_df['balanced_accuracy'].mean():.4f} ± {cv_df['balanced_accuracy'].std():.4f}")
print(f"  ROC-AUC:          {cv_df['roc_auc'].mean():.4f} ± {cv_df['roc_auc'].std():.4f}")
print(f"  Recall:           {cv_df['recall'].mean():.4f} ± {cv_df['recall'].std():.4f}")
print(f"  Precision:        {cv_df['precision'].mean():.4f} ± {cv_df['precision'].std():.4f}")


In [ ]:
# ============================================================================
# SKLEARN PREPROCESSING + MODEL TRAINING PIPELINE
# ============================================================================
print("\n" + "=" * 60)
print("BUILDING FINAL PIPELINE: PREPROCESSING + LightGBM")
print("=" * 60)

# Identify feature types
numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X_train.select_dtypes(include=['category', 'object']).columns.tolist()

print(f"Numeric features: {len(numeric_features)}")
print(f"Categorical features: {len(categorical_features)}")

# Preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ]
)

# Calculate scale_pos_weight to handle class imbalance
neg_count = (y_train == 0).sum()
pos_count = (y_train == 1).sum()
scale_pos_weight = neg_count / pos_count
print(f"\nClass imbalance adjustment:")
print(f"  Negative class (no churn): {neg_count}")
print(f"  Positive class (churn): {pos_count}")
print(f"  Scale pos weight: {scale_pos_weight:.2f}")

# Full pipeline with balanced accuracy and scale_pos_weight
model_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', lgb.LGBMClassifier(
        random_state=42,
        n_estimators=100,
        verbose=-1,
        scale_pos_weight=scale_pos_weight,  # Weight positive class more
        max_depth=6,
        learning_rate=0.1,
        min_child_weight=1,
        subsample=0.8,
        colsample_bytree=0.8,
        n_jobs=-1  # Use all processors
    ))
])

print("\nTraining model...")
model_pipeline.fit(X_train, y_train)

print("\nEvaluating on test set...")
y_pred = model_pipeline.predict(X_test)
y_pred_proba = model_pipeline.predict_proba(X_test)[:, 1]

print("\n" + "=" * 60)
print("RESULTS")
print("=" * 60)
print(classification_report(y_test, y_pred))
print(f'ROC-AUC: {roc_auc_score(y_test, y_pred_proba):.4f}')

# Calculate balanced accuracy
from sklearn.metrics import balanced_accuracy_score
balanced_acc = balanced_accuracy_score(y_test, y_pred)
print(f'Balanced Accuracy: {balanced_acc:.4f}')

In [ ]:
# ============================================================================
# FEATURE IMPORTANCE ANALYSIS
# ============================================================================
print("\n" + "=" * 60)
print("FEATURE IMPORTANCE ANALYSIS")
print("=" * 60)

# Get the trained LightGBM classifier from the pipeline
lgb_model = model_pipeline.named_steps['classifier']

# Get feature importance (split-based)
feature_importance = lgb_model.feature_importances_

# Get feature names after preprocessing
# Since we only have numeric features, they pass through StandardScaler unchanged
feature_names = numeric_features

# Create dataframe for better visualization
importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

print("\nTop 20 Most Important Features:")
print("=" * 60)
print(importance_df.head(20).to_string(index=False))

# Visualize top 15 features
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
top_n = 15
top_features = importance_df.head(top_n)

plt.barh(range(len(top_features)), top_features['importance'])
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Importance (Split Count)')
plt.title(f'Top {top_n} Most Important Features')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

# Calculate cumulative importance
importance_df['cumulative_importance'] = importance_df['importance'].cumsum() / importance_df['importance'].sum()

print(f"\nFeature Importance Summary:")
print(f"  Total features: {len(importance_df)}")
print(f"  Top 10 features account for: {importance_df.head(10)['cumulative_importance'].iloc[-1]:.2%} of importance")
print(f"  Top 20 features account for: {importance_df.head(20)['cumulative_importance'].iloc[-1]:.2%} of importance")


In [ ]:
# ============================================================================
# GENERATE SUBMISSION FILE USING TEST.PARQUET
# ============================================================================
print("\n" + "=" * 60)
print("GENERATING FINAL SUBMISSION")
print("=" * 60)

# Load test data
print("\n1. Loading test data...")
df_test_raw = pd.read_parquet(root + '/data/test.parquet')

# Clean up test data (same as raw data)
object_cols_test = df_test_raw.select_dtypes(include="object").columns
df_test_raw[object_cols_test] = df_test_raw[object_cols_test].astype("category")
df_test_raw = df_test_raw.drop(columns=['gender', 'firstName', 'lastName', 'location', 'userAgent'])

print(f"   Test raw data shape: {df_test_raw.shape}")

# Aggregate test data to user-day level
print("\n2. Aggregating test data to user-day level...")
df_test_agg = aggregate_user_day_activity(df_test_raw)
df_test_agg['userId'] = df_test_agg['userId'].astype(int)
df_test_agg['date'] = pd.to_datetime(df_test_agg['date'])

print(f"   Aggregated test shape: {df_test_agg.shape}")
print(f"   Unique users: {df_test_agg['userId'].nunique()}")

# Find the maximum date in test data
max_date = df_test_agg['date'].max()
print(f"   Max date in test data: {max_date.date()}")

# Compute rolling average features on test data
print("\n3. Computing rolling average features on test data...")
# Add 7-day rolling averages
test_with_rolling = add_rolling_averages(
    df_test_agg,
    columns=['Add Friend', 'Add to Playlist', 'Thumbs Down', 'Thumbs Up', 'Error'],
    n=7
)

# Add 14-day rolling averages
test_with_rolling = add_rolling_averages(
    test_with_rolling,
    columns=['Add Friend', 'Add to Playlist', 'Thumbs Down', 'Thumbs Up', 'Error'],
    n=14
)

# Compute thumbs ratios for both 7d and 14d
for window in [7, 14]:
    up_col = f'thumbs_up_avg_{window}d'
    down_col = f'thumbs_down_avg_{window}d'
    ratio_col = f'thumbs_ratio_{window}d'
    if up_col in test_with_rolling.columns and down_col in test_with_rolling.columns:
        denominator = test_with_rolling[up_col] + test_with_rolling[down_col]
        test_with_rolling[ratio_col] = test_with_rolling[up_col] / denominator.replace(0, np.nan)
        test_with_rolling[ratio_col] = test_with_rolling[ratio_col].fillna(0)

# Compute trend features (7d vs 14d comparison)
for col in ['Add Friend', 'Add to Playlist', 'Thumbs Down', 'Thumbs Up', 'Error']:
    col_7d = f'{col.lower().replace(" ", "_")}_avg_7d'
    col_14d = f'{col.lower().replace(" ", "_")}_avg_14d'
    trend_col = f'{col.lower().replace(" ", "_")}_trend'
    
    if col_7d in test_with_rolling.columns and col_14d in test_with_rolling.columns:
        denominator = test_with_rolling[col_14d].replace(0, np.nan)
        test_with_rolling[trend_col] = (test_with_rolling[col_7d] / denominator) - 1
        test_with_rolling[trend_col] = test_with_rolling[trend_col].fillna(0)

# Preprocess: Convert level to binary
if 'level' in test_with_rolling.columns:
    test_with_rolling['level'] = (test_with_rolling['level'] == 'paid').astype(int)

# Get the latest row for each user
last_user_data_with_rolling = test_with_rolling.sort_values('date').groupby('userId').tail(1).reset_index(drop=True)

print(f"   Users in test: {len(last_user_data_with_rolling)}")

# Ensure all feature columns from training exist in test data
print("\n4. Aligning features with training data...")
exclude_cols = ['userId', 'date', 'churn_status']
train_feature_cols = [col for col in X_train.columns]

print(f"   Training features: {len(train_feature_cols)}")
print(f"   Test columns: {len(last_user_data_with_rolling.columns)}")

# Add missing columns with 0 values if they don't exist
for col in train_feature_cols:
    if col not in last_user_data_with_rolling.columns:
        print(f"   - Adding missing column: {col}")
        last_user_data_with_rolling[col] = 0

# Select only the training feature columns and in the correct order
X_test_final = last_user_data_with_rolling[train_feature_cols].copy()

print(f"   Final feature matrix shape: {X_test_final.shape}")

# Get predictions from the trained model
print("\n5. Computing churn predictions...")
print(f"   Predicting churn window: {(max_date + pd.Timedelta(days=1)).date()} to {(max_date + pd.Timedelta(days=10)).date()}")

y_pred_final = model_pipeline.predict(X_test_final)

# Create submission: one row per unique user
submission = pd.DataFrame({
    'id': last_user_data_with_rolling['userId'].astype(int).values,
    'target': y_pred_final
})

# Save to CSV
output_path = root + '/data/submission_mdp.csv'
submission.to_csv(output_path, index=False)

print(f"\n✓ Submission saved to: {output_path}")
print(f"  Shape: {submission.shape} (one prediction per user)")
print(f"  Columns: {submission.columns.tolist()}")
print(f"\n  First 10 rows:")
print(submission.head(10))
print(f"\n  Target distribution:")
print(submission['target'].value_counts())
print(f"  Churn rate: {submission['target'].mean():.4f}")
print(f"\n  Prediction details:")
print(f"  - Latest date in test data: {max_date.date()}")
print(f"  - Predicting churn in 10 days after: {max_date.date()}")
print(f"  - Number of users: {len(submission)}")

In [ ]:
# ============================================================================
# PIPELINE ARCHITECTURE SUMMARY
# ============================================================================
#
# FEATURE ENGINEERING TRANSFORMERS (prevent data leakage):
#   1. CancellationTargetTransformer → Vectorized churn target computation
#   2. RollingAverageTransformer → 7-day rolling averages
#   3. ThumbsRatioTransformer → Derived feature (thumbs_up / thumbs_down)
#   4. FeaturePreprocessor → Type conversions & missing value handling
#
# SKLEARN PREPROCESSING (fit on train only):
#   5. ColumnTransformer → StandardScaler + OneHotEncoder
#
# MODEL:
#   6. XGBClassifier → Gradient boosting classifier
#
# KEY IMPROVEMENTS:
#   ✓ All transformations in pipeline → reproducible & maintainable
#   ✓ Vectorized churn computation → ~30x faster than looping
#   ✓ No data leakage → train & test pipelines independent
#   ✓ Time-series aware → test rolling windows use training history

print("✓ Pipeline complete with all transformations!")
print("✓ Ready for production deployment or cross-validation")

## Next Steps: Further Improvements

### 1. **Time-Series Cross-Validation**
Instead of a single train/test split, implement walk-forward validation:
- Multiple train/test splits with increasing training windows
- More robust performance estimates
- Better understanding of model stability over time

### 2. **Rolling Feature Leakage Check**
Current implementation computes rolling averages once on full dataset. Consider:
- Recomputing features separately for train and test
- Ensuring test set rolling windows only use past data

### 3. **Prediction Horizon Tuning**
Current `window_days=10` for churn prediction. Experiment with:
- 7-day window (shorter-term prediction)
- 14-day or 30-day window (longer-term prediction)
- Match to business use case requirements

### 4. **Feature Engineering**
- Add trend features (increasing/decreasing activity)
- User lifecycle stage indicators
- Recent behavior change detection
- Interaction features between subscription level and usage

### 5. **Class Imbalance Handling**
- Current churn rate: ~4.4% in train, ~1.8% in test
- Consider SMOTE or other resampling techniques
- Adjust classification threshold based on business costs